**CRNN Number Recognition**

Import needed libraries.

In [ ]:
# --- Install dependencies ---
!pip install tensorflow matplotlib --quiet

# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
import cv2
import pandas as pd
from pathlib import Path
from itertools import chain
from google.colab import drive
drive.mount('/content/drive')

Parameters and Character Mapping

In [ ]:
# --- Parameters ---
img_height, img_width = 32, 100
max_label_length = 5  # Max number length (e.g., up to 5 digits)
characters = '0123456789-'  # Add '-' if you want to recognize negative numbers
num_classes = len(characters) + 1  # +1 for CTC blank

# --- Character mapping ---
char_to_num = {c: i for i, c in enumerate(characters)}
num_to_char = {i: c for i, c in enumerate(characters)}

print("char_to_num:", char_to_num)
print("num_to_char:", num_to_char)

**Load Data**

In [ ]:
# --- Load Data ---
def load_data(image_dir, label_csv):
    df = pd.read_csv(label_csv)
    images = []
    labels = []
    for _, row in df.iterrows():
        img = cv2.imread(str(Path(image_dir) / row['filename']), cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, -1)
        images.append(img)
        labels.append(str(row['label']))
    return np.array(images), labels

X_train, y_train = load_data('/content/drive/MyDrive/NumberDetect/crnn_dataset/train/images', '/content/drive/MyDrive/NumberDetect/crnn_dataset/train/labels.csv')
X_val, y_val = load_data('/content/drive/MyDrive/NumberDetect/crnn_dataset/val/images', '/content/drive/MyDrive/NumberDetect/crnn_dataset/val/labels.csv')

# --- Encode Labels ---
def encode_labels(labels, maxlen, char_to_num):
    encoded = np.ones((len(labels), maxlen)) * -1
    for i, label in enumerate(labels):
        for j, char in enumerate(label):
            encoded[i, j] = char_to_num[char]
    return encoded

y_train_encoded = encode_labels(y_train, max_label_length, char_to_num)
y_val_encoded = encode_labels(y_val, max_label_length, char_to_num)

for i in range(5):
    print(f"Original label: {y_train[i]}")
    print(f"Encoded label: {y_train_encoded[i]}")
    # Decode back to string (ignoring -1)
    decoded = ''.join([num_to_char[int(idx)] for idx in y_train_encoded[i] if idx != -1])
    print(f"Decoded back: {decoded}")
    print("-" * 30)

**CRNN Model**

In [ ]:
# --- CRNN Model ---
def build_crnn(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    new_shape = (input_shape[1]//4, (input_shape[0]//4)*128)
    x = layers.Reshape(target_shape=new_shape)(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, x)
    return model

crnn = build_crnn((img_height, img_width, 1), num_classes)
crnn.summary()

# --- CTC Loss Layer ---
class CTCLossLayer(layers.Layer):
    def call(self, inputs):
        y_pred, labels, input_length, label_length = inputs
        loss = K.ctc_batch_cost(labels, y_pred, input_length, label_length)
        self.add_loss(loss)
        return y_pred

# --- Model for Training ---
labels = layers.Input(name='labels', shape=(max_label_length,), dtype='float32')
input_length = layers.Input(name='input_length', shape=(1,), dtype='int64')
label_length = layers.Input(name='label_length', shape=(1,), dtype='int64')
y_pred = crnn.output
ctc_out = CTCLossLayer()([y_pred, labels, input_length, label_length])
training_model = models.Model([crnn.input, labels, input_length, label_length], ctc_out)
training_model.compile(optimizer='adam')

**Training**

In [ ]:
batch_size = 32

def data_generator(X, y, batch_size=32):
    while True:
        idxs = np.random.permutation(len(X))
        for i in range(0, len(X), batch_size):
            batch_idx = idxs[i:i+batch_size]
            X_batch = X[batch_idx]
            y_batch = y[batch_idx].astype(np.int32)
            input_len = np.ones((len(X_batch), 1), dtype=np.int32) * (img_width // 4)
            label_len = np.array([[len(label[label != -1])] for label in y_batch], dtype=np.int32)
            yield (X_batch, y_batch, input_len, label_len), np.zeros(len(X_batch), dtype=np.float32)

output_signature = (
    (
        tf.TensorSpec(shape=(None, img_height, img_width, 1), dtype=tf.float32),  # X_batch
        tf.TensorSpec(shape=(None, max_label_length), dtype=tf.int32),          # y_batch
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32),                         # input_len
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32),                         # label_len
    ),
    tf.TensorSpec(shape=(None,), dtype=tf.float32)                                # dummy y
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(X_train, y_train_encoded, batch_size),
    output_signature=output_signature
)
val_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(X_val, y_val_encoded, batch_size),
    output_signature=output_signature
)

history = training_model.fit(
    train_dataset,
    steps_per_epoch=len(X_train)//batch_size,
    epochs=10,
    validation_data=val_dataset,
    validation_steps=len(X_val)//batch_size
)

**Validation**

In [ ]:
# --- Inference and Decoding ---
def decode_prediction(pred, num_to_char):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    decoded, _ = K.ctc_decode(pred, input_length=input_len, greedy=True)
    out = []
    for seq in decoded[0].numpy():
        text = ''.join([num_to_char.get(i, '') for i in seq if i != -1])
        out.append(text)
    return out

# --- Test on Validation Set ---
preds = crnn.predict(X_val[:10])
decoded = decode_prediction(preds, num_to_char)
for i in range(10):
    plt.imshow(X_val[i].squeeze(), cmap='gray')
    plt.title(f"True: {y_val[i]} | Pred: {decoded[i]}")
    plt.axis('off')
    plt.show()
    

**Save model**